In [100]:
import re
import smtplib
import logging
import socket


import dns
import socket
import smtplib
from dns import resolver

In [94]:
WSP = r'[\s]'                                        # see 2.2.2. Structured Header Field Bodies
CRLF = r'(?:\r\n)'                                   # see 2.2.3. Long Header Fields
NO_WS_CTL = r'\x01-\x08\x0b\x0c\x0f-\x1f\x7f'        # see 3.2.1. Primitive Tokens
QUOTED_PAIR = r'(?:\\.)'                             # see 3.2.2. Quoted characters
FWS = r'(?:(?:' + WSP + r'*' + CRLF + r')?' + \
      WSP + r'+)'                                    # see 3.2.3. Folding white space and comments
CTEXT = r'[' + NO_WS_CTL + \
        r'\x21-\x27\x2a-\x5b\x5d-\x7e]'              # see 3.2.3
CCONTENT = r'(?:' + CTEXT + r'|' + \
           QUOTED_PAIR + r')'                        # see 3.2.3 (NB: The RFC includes COMMENT here
# as well, but that would be circular.)
COMMENT = r'\((?:' + FWS + r'?' + CCONTENT + \
          r')*' + FWS + r'?\)'                       # see 3.2.3
CFWS = r'(?:' + FWS + r'?' + COMMENT + ')*(?:' + \
       FWS + '?' + COMMENT + '|' + FWS + ')'         # see 3.2.3
ATEXT = r'[\w!#$%&\'\*\+\-/=\?\^`\{\|\}~]'           # see 3.2.4. Atom
ATOM = CFWS + r'?' + ATEXT + r'+' + CFWS + r'?'      # see 3.2.4
DOT_ATOM_TEXT = ATEXT + r'+(?:\.' + ATEXT + r'+)*'   # see 3.2.4
DOT_ATOM = CFWS + r'?' + DOT_ATOM_TEXT + CFWS + r'?' # see 3.2.4
QTEXT = r'[' + NO_WS_CTL + \
        r'\x21\x23-\x5b\x5d-\x7e]'                   # see 3.2.5. Quoted strings
QCONTENT = r'(?:' + QTEXT + r'|' + \
           QUOTED_PAIR + r')'                        # see 3.2.5
QUOTED_STRING = CFWS + r'?' + r'"(?:' + FWS + \
                r'?' + QCONTENT + r')*' + FWS + \
                r'?' + r'"' + CFWS + r'?'
LOCAL_PART = r'(?:' + DOT_ATOM + r'|' + \
             QUOTED_STRING + r')'                    # see 3.4.1. Addr-spec specification
DTEXT = r'[' + NO_WS_CTL + r'\x21-\x5a\x5e-\x7e]'    # see 3.4.1
DCONTENT = r'(?:' + DTEXT + r'|' + \
           QUOTED_PAIR + r')'                        # see 3.4.1
DOMAIN_LITERAL = CFWS + r'?' + r'\[' + \
                 r'(?:' + FWS + r'?' + DCONTENT + \
                 r')*' + FWS + r'?\]' + CFWS + r'?'  # see 3.4.1
DOMAIN = r'(?:' + DOT_ATOM + r'|' + \
         DOMAIN_LITERAL + r')'                       # see 3.4.1
ADDR_SPEC = LOCAL_PART + r'@' + DOMAIN               # see 3.4.1

# A valid address will match exactly the 3.4.1 addr-spec.
VALID_ADDRESS_REGEXP = '^' + ADDR_SPEC + '$'

## VALIDACION DE SINTAXIS

In [95]:
def validate_syntax(email):
    try:
        assert re.match(VALID_ADDRESS_REGEXP, email) is not None
    except AssertionError:
        return False
    
    return True

In [96]:
validate_syntax('dcsdcds@c.com')

True

### VALIDACION DE HOST

In [158]:
host_mx_dict = {}
def validate_hostname(hostname):
    try:
        if hostname in host_mx_dict:
            return host_mx_dict[hostname]
        
        intent = 1
        first = str(resolver.resolve(hostname, "MX")[0].exchange)
        
        while first == 'errdomain.' and intent < 100:
            first = str(resolver.resolve(hostname, "MX")[0].exchange)
            intent += 1
        
        print("intentos: ", intent)
        host_mx_dict[hostname] = first
        return first
    except Exception as e:
        print(str(e))
        return False
    
mx_record = validate_hostname('gmail.com')
print(mx_record)

intentos:  8
alt4.gmail-smtp-in.l.google.com.


In [159]:
validate_hostname('gmaicdscl.com')

The DNS query name does not exist: gmaicdscl.com.


False

In [160]:
host_mx_dict

{'google.com': 'alt1.aspmx.l.google.com.',
 'gmail.com': 'alt4.gmail-smtp-in.l.google.com.'}

### VALIDACION DE EMAILS

In [173]:
host = socket.gethostname()
server = smtplib.SMTP()
server.set_debuglevel(0)

server.connect(mx_record)
server.helo(host)
server.mail('me@gmail.com')
code, message = server.rcpt('v.villacorta.unmsm@gmail.com')
server.quit()

# Assume 250 as Success
if code == 250:
	print(True)
else:
	print(False)

True


In [162]:
help(server.rcpt)

Help on method rcpt in module smtplib:

rcpt(recip, options=()) method of smtplib.SMTP instance
    SMTP 'rcpt' command -- indicates 1 recipient for this mail.

